1. 데이터 로드 및 기본 정보 확인
텍스트 데이터를 불러와서 크기, 샘플, 결측치 여부 등을 확인합니다.

In [ ]:
import pandas as pd

# 예시: 판례 데이터를 포함하는 CSV 파일 불러오기
df = pd.read_csv('court_cases.csv')

# 데이터의 크기 확인
print(f"데이터 크기: {df.shape}")

# 데이터의 상위 5개 샘플 확인
print(df.head())

# 결측치 확인
print(df.isnull().sum())


2. 텍스트 길이 분포
텍스트 데이터의 길이(단어 수 또는 글자 수)를 확인하면, 텍스트 데이터의 범위를 이해하고, 모델의 입력으로 사용될 적절한 시퀀스 길이를 결정하는 데 도움이 됩니다.

In [ ]:
import matplotlib.pyplot as plt

# 각 텍스트의 길이(단어 수) 계산
df['text_length'] = df['text'].apply(lambda x: len(x.split()))

# 히스토그램으로 텍스트 길이 분포 확인
plt.figure(figsize=(10, 6))
plt.hist(df['text_length'], bins=50, color='skyblue')
plt.title('텍스트 길이 분포 (단어 수 기준)')
plt.xlabel('단어 수')
plt.ylabel('문서 수')
plt.show()


3. 단어 빈도 분석
데이터에서 자주 사용되는 단어를 확인하면, 특정 단어가 텍스트 데이터에서 차지하는 비중을 알 수 있으며, 중요한 키워드를 발견하는 데 유용합니다.

In [ ]:
from collections import Counter
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# 불용어 설정
stop_words = set(stopwords.words('korean'))

# 모든 텍스트에서 단어 추출 및 빈도 계산
all_words = ' '.join(df['text']).split()
filtered_words = [word for word in all_words if word not in stop_words]
word_freq = Counter(filtered_words)

# 상위 10개 자주 등장하는 단어 확인
common_words = word_freq.most_common(10)
print("자주 등장하는 단어:", common_words)


In [ ]:
# 한국어

from konlpy.tag import Okt
from collections import Counter

okt = Okt()

# 한국어 텍스트에서 명사 추출
df['nouns'] = df['text'].apply(lambda x: okt.nouns(x))

# 모든 문서에서 추출한 명사를 합침
all_words = [word for words in df['nouns'] for word in words]

# 단어 빈도 계산
word_freq = Counter(all_words)

# 상위 10개 자주 등장하는 단어 확인
common_words = word_freq.most_common(10)
print("자주 등장하는 단어:", common_words)


4. WordCloud 시각화
단어 빈도 분석을 바탕으로 자주 등장하는 단어를 시각화하여 직관적으로 데이터를 이해할 수 있습니다.

In [ ]:
from wordcloud import WordCloud

# 단어 빈도 기반 워드클라우드 생성
wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_freq)

# 워드클라우드 시각화
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()


In [ ]:
# 한국어

from wordcloud import WordCloud

# 한국어 글꼴을 지정하여 워드클라우드 생성 (글꼴 경로는 시스템에 맞게 수정)
wordcloud = WordCloud(font_path='/Library/Fonts/AppleGothic.ttf', width=800, height=400, background_color='white').generate_from_frequencies(word_freq)

plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()


5. 클래스 불균형 확인
분류 작업일 경우, 각 클래스(예: 판결 결과, 적용 법률)의 불균형 여부를 확인하는 것이 중요합니다. 특정 클래스에 편향되지 않도록 데이터를 처리해야 하기 때문입니다

In [ ]:
# 클래스 분포 확인 (예: 'label' 컬럼에 판결 결과가 있다고 가정)
df['label'].value_counts().plot(kind='bar', color='skyblue')
plt.title('클래스 분포')
plt.xlabel('클래스')
plt.ylabel('문서 수')
plt.show()

6. N-그램 분석
N-그램 분석은 단어 쌍(또는 세 단어)의 패턴을 찾는 데 유용합니다. 예를 들어, 두 단어 이상이 자주 함께 등장하는지 확인할 수 있습니다

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# 2-그램(바이그램) 분석
vectorizer = CountVectorizer(ngram_range=(2, 2), stop_words=stop_words)
X2 = vectorizer.fit_transform(df['text'])

# 상위 10개의 자주 등장하는 바이그램 출력
bi_grams = vectorizer.get_feature_names_out()
bi_gram_counts = X2.toarray().sum(axis=0)
bi_gram_freq = dict(zip(bi_grams, bi_gram_counts))
sorted_bi_grams = sorted(bi_gram_freq.items(), key=lambda x: x[1], reverse=True)[:10]
print("상위 10개 바이그램:", sorted_bi_grams)


In [ ]:
# 한국어

from sklearn.feature_extraction.text import CountVectorizer

# 한국어 텍스트를 형태소 분석으로 나눈 후, 이를 이용해 2-그램 분석
df['tokenized_text'] = df['text'].apply(lambda x: ' '.join(okt.morphs(x)))

vectorizer = CountVectorizer(ngram_range=(2, 2))  # 바이그램
X2 = vectorizer.fit_transform(df['tokenized_text'])

# 상위 10개의 자주 등장하는 바이그램 출력
bi_grams = vectorizer.get_feature_names_out()
bi_gram_counts = X2.toarray().sum(axis=0)
bi_gram_freq = dict(zip(bi_grams, bi_gram_counts))
sorted_bi_grams = sorted(bi_gram_freq.items(), key=lambda x: x[1], reverse=True)[:10]
print("상위 10개 바이그램:", sorted_bi_grams)


7. 단어의 상관관계 분석
단어 간의 상관관계를 분석하면 특정 단어가 함께 나타나는 패턴을 발견할 수 있습니다. 상관관계는 문맥 상 단어의 관계를 파악하는 데 유용합니다.

In [ ]:
import seaborn as sns
import numpy as np

# 샘플 단어 목록
words_to_check = ['법률', '판례', '형법', '민법']

# 단어 상관관계 계산을 위한 데이터 프레임 생성
word_matrix = np.zeros((len(df), len(words_to_check)))

for i, text in enumerate(df['text']):
    for j, word in enumerate(words_to_check):
        word_matrix[i, j] = text.split().count(word)

# 단어 간 상관관계 계산
correlation_matrix = np.corrcoef(word_matrix.T)

# 상관관계 히트맵 시각화
sns.heatmap(correlation_matrix, annot=True, xticklabels=words_to_check, yticklabels=words_to_check)
plt.title('단어 간 상관관계')
plt.show()


In [ ]:
# 한국어

import numpy as np
import seaborn as sns

# 단어 상관관계를 분석할 단어 리스트
words_to_check = ['법률', '판례', '형법', '민법']

# 각 단어가 문서에서 얼마나 자주 등장하는지 카운팅
word_matrix = np.zeros((len(df), len(words_to_check)))

for i, text in enumerate(df['text']):
    for j, word in enumerate(words_to_check):
        word_matrix[i, j] = text.count(word)

# 단어 간 상관관계 계산
correlation_matrix = np.corrcoef(word_matrix.T)

# 히트맵으로 상관관계 시각화
sns.heatmap(correlation_matrix, annot=True, xticklabels=words_to_check, yticklabels=words_to_check)
plt.title('단어 간 상관관계')
plt.show()


8. 텍스트 분포 및 중복 확인
텍스트가 반복되거나 중복된 문서가 있는지 확인하는 것이 필요합니다. 중복된 데이터가 너무 많으면 학습에 부정적인 영향을 미칠 수 있습니다.

In [ ]:
# 중복된 텍스트 확인
duplicates = df[df.duplicated(subset='text', keep=False)]
print(f"중복된 문서 수: {duplicates.shape[0]}")


9. 감정 분석 또는 주제 모델링
판례나 소장 데이터를 기반으로 특정 감정(긍정, 부정) 또는 주제(상속, 형사 사건 등)를 파악할 수 있습니다. 이는 주제 모델링 또는 감정 분석으로 처리합니다.

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 기반 벡터화
tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words=stop_words)
X_tfidf = tfidf_vectorizer.fit_transform(df['text'])

# LDA를 통한 주제 모델링
lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(X_tfidf)

# 각 문서에 대한 주제 분포 확인
topics = lda.transform(X_tfidf)
print("문서별 주요 주제 분포:", topics)
